In [3]:
import os

train_images_path = "../Data/Train/"
image_folders = os.listdir(train_images_path)

for folder in image_folders:
    image_files = os.listdir(train_images_path+folder+"/")
    i = 0
    for image_file in image_files:
        ## os.rename(train_images_path+folder+"/"+image_file, train_images_path+folder+"/"+ folder + "_" + str(i) + ".jpg")
        i +=1

In [21]:
import pandas as pd

labels = []
img_paths = []
train_images_path = "../Data/Train/"
image_folders = os.listdir(train_images_path)

for folder in image_folders:
    image_files = os.listdir(train_images_path+folder+"/")

    for image_file in image_files:
        img_paths.append(folder +"/" + image_file)
        labels.append(folder)
        
df = pd.DataFrame(zip(img_paths, labels), columns=["Image", "Label"])
df.to_csv('train_data.csv', header=False, index=False)

In [80]:
import pandas as pd
from torch.utils.data import Dataset
import cv2 as cv
import numpy as np
import os

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

class ColorImageDataset(Dataset):
    def __init__(self, img_labels, dataset_dir, transform=None) -> None:
        self.img_labels = img_labels
        self.dataset_dir = dataset_dir
        self.transfor = transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, index) -> None:
        img_path = os.path.join(self.dataset_dir, self.img_labels.iloc[index, 0])
        label = self.img_labels.iloc[index, 1]
        img = cv.imread(img_path) # opencv reads in bgr order
        mu = np.mean(img.reshape(3, -1), axis=1, dtype=np.float64)/255 # take the mean in every chanel for the first RGB moment (normalized)
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV) 
        img_hsv = img_hsv.astype(np.float64)
        img_hsv[:, :, 0] = np.cos(np.pi * img_hsv[:, :, 0]/180)
        ## hist = cv.calcHist([img_hsv], channels=[0, 1], mask=None, histSize=[8, 8], ranges=[[-1, 1], [0, 256]])
        hist_h = np.histogram(img_hsv[:, :, 0], bins=8, range=[-1, 1])
        hist_s = np.histogram(img_hsv[:, :, 1], bins=8, range=[0, 256])

        hist_h = softmax(np.asarray(hist_h[0], dtype=np.float64))
        hist_s = softmax(np.asarray(hist_s[0], dtype=np.float64))
        return np.concatenate([mu, hist_h, hist_s]), label


        


In [82]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

label_file = "train_data.csv"
dataset_dir = "../Data/Train"
data = pd.read_csv(label_file)

train_df, valid_df = train_test_split(data, test_size=0.1, random_state=42)    

train_dataset = ColorImageDataset(img_labels=train_df, dataset_dir=dataset_dir)
valid_dataset = ColorImageDataset(img_labels=valid_df, dataset_dir=dataset_dir)



In [85]:
sample, label = valid_dataset[0]
print(sample)
print(label)

sample, label = valid_dataset[1]
print(sample)
print(label)

[0.49232547 0.49545365 0.45116226 0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 1.        ]
Blue
[0.6226911  0.62285056 0.80863195 1.         0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.         0.         0.         0.         0.
 0.        ]
Violet


In [91]:
train_list_data = [data.tolist() for data, _ in train_dataset]
train_list_label = [label for _, label in train_dataset]
print(train_list_data[0])

df_data = pd.DataFrame(train_list_data)
df_label = pd.DataFrame(train_list_label, columns=["Labels"])

[0.7030394256092561, 0.5149564163607233, 0.5917027034451731, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.98937865328318e-301, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [95]:
valid_list_data = [data.tolist() for data, _ in valid_dataset]
valid_list_label = [label for _, label in valid_dataset]
print(train_list_data[0])

df_valid_data = pd.DataFrame(valid_list_data)
df_valid_label = pd.DataFrame(valid_list_label, columns=["Labels"])

[0.7030394256092561, 0.5149564163607233, 0.5917027034451731, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.98937865328318e-301, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [96]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(df_data)
X_test = scaler.transform(df_valid_data)



In [97]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02,2.870000e+02
mean,2.970910e-16,-4.146896e-16,4.085002e-16,-2.475759e-17,3.094698e-17,-1.856819e-17,-3.713638e-17,-3.713638e-17,3.558903e-17,3.868373e-17,6.808336e-17,-2.785228e-17,3.713638e-17,3.713638e-17,1.547349e-17,-2.475759e-17,3.094698e-18,6.189397e-18,-1.237879e-17
std,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00,1.001747e+00
min,-2.530297e+00,-2.399633e+00,-2.244489e+00,-3.156214e-01,-2.809757e-01,-3.416599e-01,-2.429827e-01,-1.027783e-01,-1.581139e-01,-2.088932e-01,-1.222969e+00,-9.623796e-01,-2.088932e-01,-1.581139e-01,-1.799280e-01,-1.693335e-01,-2.088932e-01,-2.736903e-01,-6.270597e-01
25%,-6.530601e-01,-6.458626e-01,-7.162007e-01,-3.156214e-01,-2.809757e-01,-3.416599e-01,-2.429827e-01,-1.027783e-01,-1.581139e-01,-2.088932e-01,-1.222969e+00,-9.623796e-01,-2.088932e-01,-1.581139e-01,-1.799280e-01,-1.693335e-01,-2.088932e-01,-2.736903e-01,-6.270597e-01
50%,6.389898e-02,2.756049e-02,1.127758e-02,-3.156214e-01,-2.809757e-01,-3.416599e-01,-2.429827e-01,-1.027783e-01,-1.581139e-01,-2.088932e-01,8.176825e-01,-9.623796e-01,-2.088932e-01,-1.581139e-01,-1.799280e-01,-1.693335e-01,-2.088932e-01,-2.736903e-01,-6.270597e-01
75%,7.651614e-01,5.733445e-01,7.322446e-01,-3.156214e-01,-2.809757e-01,-3.416599e-01,-2.429827e-01,-1.027783e-01,-1.581139e-01,-2.088932e-01,8.176825e-01,1.039091e+00,-2.088932e-01,-1.581139e-01,-1.799280e-01,-1.693335e-01,-2.088932e-01,-2.736903e-01,1.594744e+00
max,1.812087e+00,2.356865e+00,2.100984e+00,3.168353e+00,3.559026e+00,2.926887e+00,4.115519e+00,9.729680e+00,6.324555e+00,4.787136e+00,8.176825e-01,1.039091e+00,4.787136e+00,6.324555e+00,5.557777e+00,5.905506e+00,4.787136e+00,3.653765e+00,1.594744e+00


In [126]:
from sklearn.svm import SVC


# import metrics to compute accuracy
from sklearn.metrics import accuracy_score


# instantiate classifier with default hyperparameters
svc=SVC(kernel='poly', C=30, degree=3) 

print(df_label.head())
# fit classifier to training set
svc.fit(X_train,df_label["Labels"])



   Labels
0     Red
1   White
2  Yellow
3    Blue
4     Red


SVC(C=30, kernel='poly')

In [127]:
y_pred=svc.predict(df_valid_data)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(df_valid_label, y_pred)))

Model accuracy score with default hyperparameters: 0.5312


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(3)
knn.fit(X_train,df_label["Labels"])

y_pred=knn.predict(df_valid_data)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(df_valid_label, y_pred)))